In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
AZURE_KEY = "API_KEY::URL"
OPENAI_KEY = "sk-XXX"

## Use Azure and OpenAI models

In [ ]:
from manifest import Manifest
from manifest.connections.client_pool import ClientConnection
from pathlib import Path

cache_path = Path("manifest.db")
if cache_path.exists():
    cache_path.unlink()


azure = ClientConnection(
    client_name="azureopenai",
    client_connection=AZURE_KEY,
    engine="text-davinci-003",
)

manifest = Manifest(client_pool=[azure], 
    cache_name="sqlite",
    cache_connection="manifest.db"
)


openai = ClientConnection(
    client_name="openai",
    client_connection=OPENAI_KEY,
    engine="text-davinci-003",
)

manifest_openai_nocache = Manifest(client_pool=[openai])

manifest_openai = Manifest(client_pool=[openai], 
    cache_name="sqlite",
    cache_connection="manifest.db"
)

In [ ]:
# Show caches are the same
text = "What is the meaning of life?"
res = manifest.run(text, max_tokens=100, temperature=0.7, return_response=True)
print(res.get_response())
print(res.is_cached())
res2 = manifest_openai.run(text, max_tokens=100, temperature=0.7, return_response=True)
print(res2.is_cached())

assert res2.get_response() == res.get_response()

In [ ]:
azure_chat = ClientConnection(
    client_name="azureopenaichat",
    client_connection=AZURE_KEY,
    engine="gpt-3.5-turbo",
)

manifest = Manifest(client_pool=[azure_chat])

In [ ]:
print(manifest.run("What do you think is the best food?", max_tokens=100))

chat_dict = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
]
print(manifest.run(chat_dict, max_tokens=100))